In [ ]:
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS



In [246]:
text_splitter = CharacterTextSplitter(       
    separator = "####",
    chunk_size = 300,
    chunk_overlap  = 4,
    length_function = len,
    is_separator_regex = False,
)

In [248]:
with open("headphone_new.txt", 'r', encoding='utf-8') as f:
    real_estate_sales = f.read()

In [249]:
docs = text_splitter.create_documents([real_estate_sales])


In [257]:
docs[0]

Document(page_content='品牌: OnePlus\nheadphone name: OnePlus Bullets Z2 Bluetooth Wireless in Ear Earphones with Mic\nWireless: yes\n续航时间: 20 Hrs\n充电时间: 10 Mins\nmode: Bombastic Bass - 12.4 Mm Drivers\n价格: 299', metadata={})

In [258]:
len(docs)


50

In [254]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [255]:
db.save_local("headphone_new")

In [271]:
query = "有没有无线耳机"
answer_list = db.similarity_search(query)
for doc in answer_list:
    print(doc.page_content + "\n")

品牌: Noise
headphone name: Noise Buds VS104 in-Ear Truly Wireless Earbuds with 30H of Playtime, Instacharge(10 min=150 min), 13mm Driver, Mic, Hyper
是否无线耳机: 无线
续航时间: 30H
充电时间: Instacharge(10 min=150 min)
mode: 13mm Driver, Mic, Hyper
价格: 499

品牌: Oppo
headphone name: Oppo Enco M32 Bluetooth Wireless in Ear Earbuds with Mic
是否无线耳机: 无线
续航时间: 28Hrs
充电时间: 10 Mins Charge - 20Hrs Music Fast Charge
mode: 10mm
价格: 999

品牌: OnePlus
headphone name: OnePlus Bullets Z2 Bluetooth Wireless in Ear Earphones with Mic
是否无线耳机: 无线
续航时间: 20 Hrs
充电时间: 10 Mins
mode: Bombastic Bass - 12.4 Mm Drivers
价格: 299

品牌: boAt
headphone name: boAt Bassheads 105 Wired in Ear Earphones with Mic (Black)
是否无线耳机: 有线
续航时间: N/A
充电时间: N/A
mode: N/A
价格: 99



In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.load_local("headphone_new", OpenAIEmbeddings())


In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
db = FAISS.load_local("headphone_new", OpenAIEmbeddings())


In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [7]:
qa_chain({"query": query})

NameError: name 'query' is not defined

In [411]:
def get_chat_history123(inputs) -> str:
    res = []
    for human, ai in inputs:
        res.append(f"Human:{human}\nAI:{ai}")
    return "\n".join(res)


In [42]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
 

# Build prompt
template =  """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
同时请考虑以下这点
1:这个是个中文环境，当rephrase的时候，也要考虑中文的语境
"""

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["chat_history","question"], template=template)
 
qa = ConversationalRetrievalChain.from_llm(llm, db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}),
                                           verbose=True,
                                           return_generated_question=True,
                                           condense_question_prompt=QA_CHAIN_PROMPT
                                          # combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT}
                                          )

chat_history = []
query = "有没有JBL的耳机？"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
品牌: JBL
headphone name: JBL C50HI, Wired in Ear Headphones with Mic, One Button Multi-Function Remote, Lightweight & Comfortable fit (Blue)
是否无线耳机: 有线
续航时间: N/A
充电时间: N/A
mode: N/A
价格: 99

品牌: JBL
headphone name: JBL C50HI, Wired in Ear Headphones with Mic, One Button Multi-Function Remote, Lightweight & Comfortable fit (Black)
是否无线耳机: 有线
续航时间: N/A
充电时间: N/A
mode: N/A
价格: 99
Human: 有没有JBL的耳机？

> Finished chain.

> Finished chain.


'有，JBL有一款有线耳机名为JBL C50HI，售价为99元。'

In [39]:
chat_history = [(query, result["answer"])]
print(chat_history)
query = "请列一下大概500元的耳机？"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]


[('有没有JBL的耳机？', '有，JBL有一款有线耳机名为JBL C50HI，售价为99元。')]


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:

Human: 有没有JBL的耳机？
Assistant: 有，JBL有一款有线耳机名为JBL C50HI，售价为99元。
Follow Up Input: 请列一下大概500元的耳机？
Standalone question:
同时请考虑以下这点
1:这个是个中文环境，当rephrase的时候，也要考虑中文的语境


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Human: 请列一下大约500元的耳机有哪些？

> Finished chain.

> Finished chain.


'以下是大约500元左右的一些耳机选项：\n\n1. Sony MDR-XB55AP：这是一款入门级的耳机，具有强调低音的特点，适合喜欢重低音的用户。\n\n2. JBL T450BT：这是一款无线蓝牙耳机，具有舒适的耳罩设计和清晰的音质。\n\n3. Sennheiser HD 206：这是一款经济实惠的耳机，具有良好的音质和舒适的佩戴感。\n\n4. Audio-Technica ATH-M20x：这是一款专业监听耳机，适合音乐制作和录音室使用。\n\n5. AKG K92：这是一款平衡的耳机，具有清晰的音质和舒适的佩戴感。\n\n请注意，耳机的价格和型号可能会因地区和销售渠道而有所不同。建议您在购买前进行更多的市场调查和比较，以找到最适合您需求和预算的耳机。'

In [46]:
chat_history = [(query, result["answer"])]
print(chat_history)
query = "有没有JBL的耳机？"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]


[('is there JBL headphone？', '是的，我们有JBL的耳机。')]


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:

Human: is there JBL headphone？
Assistant: 是的，我们有JBL的耳机。
Follow Up Input: 有没有JBL的耳机？
Standalone question:
同时请考虑以下这点
1:这个是个中文环境，当rephrase的时候，也要考虑中文的语境


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
品牌: JBL
headphone name: JBL C50HI, Wired in Ear Headphones with Mic, One Button Multi-Function Remote, Lightweight & Comfortable fit (Blue)
是否无线耳机: 有线
续航时间: N/A
充电时间: N/A
mode: N/A
价格: 99

品牌: JBL
headphone name: JBL C50HI, Wired in Ear Headphones with Mic, One Button Mult

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-7bh0JDOeBMdvHwH3KeKISO3U on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-7bh0JDOeBMdvHwH3KeKISO3U on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.

> Finished chain.


'有，JBL有一款有线耳机名为JBL C50HI，售价为99元。'

In [301]:
print(chat_history)

[]


In [302]:
chat_history = [(query, result["answer"])]
print(chat_history)

[('有JBL的耳机吗', '是的，我们有JBL的耳机。')]


In [34]:
query = "请列一下大概500元的耳机"
chat_history = []
result = qa({"question": query,"chat_history":chat_history})
result["answer"]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Human: 请列一下大概500元的耳机

> Finished chain.

> Finished chain.


'以下是一些大约500元左右的耳机选项：\n\n1. Sony MDR-XB55AP：这是一款入门级的耳机，具有出色的低音效果和舒适的佩戴感。\n2. JBL T450BT：这是一款无线蓝牙耳机，具有清晰的声音和长时间的电池寿命。\n3. Sennheiser HD 4.20s：这是一款高保真耳机，具有出色的音质和耐用性。\n4. Audio-Technica ATH-M20x：这是一款专业监听耳机，适合音乐制作和录音室使用。\n5. Beats EP：这是一款时尚的耳机，具有强调低音的特点，适合喜欢流行音乐的用户。\n\n请注意，耳机的价格可能会因地区和销售渠道而有所不同。建议在购买前查看当地的价格和评论，以选择最适合您的耳机。'